In [13]:
# Para usar o sleep:
import time
# Para logs:
from tqdm import tqdm
import logging
import sys
sys.path.append('../utils')
from utils import get_logger, Clock, load_json, export_json
# Para web scraping:
import requests
from bs4 import BeautifulSoup, SoupStrainer
# Para acessar as credenciais de login de um arquivo externo:
import os
import dotenv
# Para navegar na página além do botão 'Show more':
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

In [3]:
base_url = 'https://untappd.com'
headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0"
}
payload = {'country': 'brazil'}

page = requests.get(base_url + '/beer/top_rated', params=payload, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

#print(soup.prettify())

In [4]:
logger.info("Coletando links de cervejas brasileiras do 'top rated'...")
brazilian_beer_links = [a_tag.get('href') for a_tag in soup.select('div.beer-item > a:first-child')]
logger.info("Feito.")
brazilian_beer_links

2023-11-18 18:52:38,518 - INFO - Coletando links de cervejas brasileiras do 'top rated'...
2023-11-18 18:52:38,545 - INFO - Feito.


['/b/spartacus-brewing-iii-three/4283069',
 '/b/spartacus-brewing-v-five/5285289',
 '/b/spartacus-brewing-iv-four/4756418',
 '/b/hocus-pocus-overdrive/1682105',
 '/b/spartacus-brewing-arete-tomo-2/4542383',
 '/b/salvador-brewing-co-cookie-de-chocolate/2908960',
 '/b/spartacus-brewing-etera-tomo-5/4769967',
 '/b/hocus-pocus-elephants-graveyard/2769185',
 '/b/spartacus-brewing-arete-tomo-1/4098457',
 '/b/cervejaria-dogma-rizoma/1536694',
 '/b/hocus-pocus-supersymmetry/1930404',
 '/b/koala-san-brew-cosmic-blur/3452440',
 '/b/trilha-cervejaria-pao-de-mel/3252352',
 '/b/hocus-pocus-rabbit-hole/2321900',
 '/b/spartacus-brewing-fwiw/3905760',
 '/b/spartacus-brewing-barrel-aged-icon-double-chocolate-peanut-butter/5076259',
 '/b/cervejaria-dogma-mosaic-lover/1698497',
 '/b/salvador-brewing-co-engesa-oil-coffee-edition/3036359',
 '/b/mad-dwarf-brewery-md-oloroso-2017/2957221',
 '/b/5-elementos-cervejaria-artesanal-coffee-and-pancake-brunch-stout/2546467',
 '/b/cervejaria-dogma-morning-gringo/265

In [5]:
number_of_brazilian_beer_links = len(brazilian_beer_links)
logger.info(f"Número total de cervejas no 'top rated' do Brasil: {number_of_brazilian_beer_links}")

2023-11-18 18:52:41,774 - INFO - Número total de cervejas no 'top rated' do Brasil: 50


### Acessando as páginas completas das cervejas com Selenium:

Carregando as credenciais de acesso:

In [9]:
dotenv.load_dotenv()
EMAIL = os.getenv('EMAIL')
PASSWORD = os.getenv('PASSWORD')

Definindo as opções do navegador que o Selenium usará:

In [10]:
options = Options()
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

Acessando a página completa de cada cerveja do _top rated_ do Brasil clicando no link 'Show more' e minerando os IDs de usuários:

In [15]:
browser = webdriver.Chrome(options=options)

browser.get(base_url + '/login')
browser.find_element(By.NAME,"username").send_keys(EMAIL)
browser.find_element(By.NAME,"password").send_keys(PASSWORD)
time.sleep(30)

interactions = []
logger.info("Iniciando uma sessão...")
with requests.Session() as session:
    logger.info("A iteração começará agora...")
    iteration_time = Clock("Coleta das infos das interações")
    with tqdm(total=number_of_brazilian_beer_links) as progress_bar:
        progress_bar.set_description("Coletando os comentários das top 50 cervejas melhor avaliadas do Brasil")
        for beer_url in brazilian_beer_links:
                try:
                    page = requests.get(base_url + beer_url, headers=headers)
                    soup = BeautifulSoup(page.text, 'html.parser')

                    # coletando comentários

                    # a_tag = SoupStrainer('a')
                    # soup = BeautifulSoup(full_page, 'html.parser', parse_only=a_tag)
                    # user = set('/user/' + link.get('href').split('/user/')[1].split('/')[0])

                    user = soup.select('div.avatar-holder > a')[0].get_text()
                    comment = soup.select('div.checkin-comment > p.comment-text')[0].get_text()

                    interaction = {'user': user, 'comment': comment}
                    interactions.append(interaction)

                    progress_bar.update(1)
                except IndexError:
                    print(f"Elemento não encontrado para a cerveja: {beer_url}")
                    progress_bar.update(1)
                    continue
        iteration_time.stop_watch()
    logger.info(f"Operação concluída! A lista filtrada possui {len(interactions)} cervejas.")
    
    print(interactions)

2023-11-18 19:08:34,308 - INFO - Iniciando uma sessão...
2023-11-18 19:08:34,310 - INFO - A iteração começará agora...
Coletando os comentários das top 50 cervejas melhor avaliadas do Brasil: 100%|██████████| 50/50 [01:03<00:00,  1.05it/s]2023-11-18 19:09:37,867 - INFO - Coleta das infos das interações levou 01m03s para ser executado.
Coletando os comentários das top 50 cervejas melhor avaliadas do Brasil: 100%|██████████| 50/50 [01:03<00:00,  1.27s/it]
2023-11-18 19:09:37,869 - INFO - Operação concluída! A lista filtrada possui 50 cervejas.


[{'user': '\n\n', 'comment': '\nLCI PITFEST '}, {'user': '\n\n', 'comment': '\nSuch a nice stout. Chocolate and vanilla. Thick and smooth. '}, {'user': '\n\n', 'comment': '\n4.5+ dopjes '}, {'user': '\n\n', 'comment': '\nDia de revisitar um clássico da cena nacional! Vou manter a nota pelo apego e pelo o que representa. Quase 7 anos depois da primeira vez. '}, {'user': '\n\n', 'comment': '\nTachtig🏎Tomo....d8 het niet. Wow wat een top🍺 uit 🇧🇷gebrouwen bij Bax in Grunn #Beerdome🍻😋💪🥳✅️ '}, {'user': '\n\n', 'comment': '\n好喝 '}, {'user': '\n\n', 'comment': '\nSPECTACULAR '}, {'user': '\n\n', 'comment': '\nLCI '}, {'user': '\n\n', 'comment': '\nTotalement Bourbon !!!! Barrel aged, vanille gourmande ! Une très belle découverte ! Merci Bapt P. !! Félicitations pour ta 5000eme !! '}, {'user': '\n\n', 'comment': '\nUm clássico sempre é um clássico. '}, {'user': '\n\n', 'comment': '\nGreat memories of this beer at the Patron open beer fest '}, {'user': '\n\n', 'comment': '\nMelhor RIS que já beb